<a href="https://colab.research.google.com/github/adipai/data-decent/blob/main/src/automation/automation_ambari_vuln_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sdv

In [ ]:
!unzip data.zip

In [ ]:
# All imports here
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
import math
import warnings

from scipy.io import arff
from sdv.datasets.local import load_csvs
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.lib.utils import display_bayesian_network
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

In [ ]:
warnings.filterwarnings('ignore')

## Data preprocessing

In [ ]:
def preprocess_data_train(X_train):
    # Count missing values before handling missing data
    missing_before = np.isnan(X_train).sum()

    # Handle missing data
    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(X_train)

    # Count missing values after handling missing data
    missing_after = np.isnan(X_train).sum()

    # Normalize numeric columns
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return X_train, scaler, imputer

def preprocess_data_test(X_test, scaler, imputer):
    # Count missing values before handling missing data
    missing_before = np.isnan(X_test).sum()

    # Handle missing data
    X_test = imputer.transform(X_test)

    # Count missing values after handling missing data
    missing_after = np.isnan(X_test).sum()

    # Normalize numeric columns
    X_test = scaler.transform(X_test)

    return X_test

## Experiments

### Dataset: Eclipse JDT

In [ ]:
train_data_path = f"data/Bug_Reports/ambari-train.csv"
test_data_path = f"data/Bug_Reports/ambari-test.csv"
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)
X_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]

## Training and testing using ML models

In [ ]:
# Generic function to test synthetic data using LR, SVM, DT

def evaluate_models(X_train, X_test, y_train, y_test, random_state=42):

    # Initialize classifiers
    classifiers = {
        "Logistic Regression": LogisticRegression(random_state=random_state),
        "SVM": SVC(random_state=random_state),
        "Decision Tree": DecisionTreeClassifier(random_state=random_state)
    }

    # Results dictionary to store evaluation metrics
    results = {}

    # Iterate over classifiers
    for name, clf in classifiers.items():
        # Fit classifier
        clf.fit(X_train, y_train)

        # Predictions
        y_pred = clf.predict(X_test)

        # Evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # AUC-ROC
        if hasattr(clf, "predict_proba"):
            y_prob = clf.predict_proba(X_test)[:,1]
        else:
            y_prob = clf.decision_function(X_test)
        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Store results
        results[name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc,
            "Confusion Matrix": cm
        }

        # Plot AUC-ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{name} - AUC-ROC Curve')
        plt.legend(loc='lower right')
        plt.savefig(f'{name}_auc_roc_curve.png', dpi=300)
        plt.close()

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'{name} - Confusion Matrix')
        plt.savefig(f'{name}_confusion_matrix.png', dpi=300)
        plt.close()

    return results

In [ ]:
results = evaluate_models(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(results)

{'Logistic Regression': {'Accuracy': 0.978, 'Precision': 0.25, 'Recall': 0.2857142857142857, 'F1 Score': 0.26666666666666666, 'ROC AUC': 0.755722978846711, 'Confusion Matrix': array([[487,   6],
       [  5,   2]])}, 'SVM': {'Accuracy': 0.986, 'Precision': 0.0, 'Recall': 0.0, 'F1 Score': 0.0, 'ROC AUC': 0.6734279918864097, 'Confusion Matrix': array([[493,   0],
       [  7,   0]])}, 'Decision Tree': {'Accuracy': 0.968, 'Precision': 0.09090909090909091, 'Recall': 0.14285714285714285, 'F1 Score': 0.1111111111111111, 'ROC AUC': 0.5612865835989568, 'Confusion Matrix': array([[483,  10],
       [  6,   1]])}}


# RRP Over-sampling

In [ ]:
def cluster(candidates, enough, res):
    if len(candidates) < enough:
        res.append(candidates)
        return res

    east, west, east_items, west_items = split(candidates)
    res = cluster(east_items, enough, res)
    res = cluster(west_items, enough, res)

    return res


def split(candidates):
    pivot = random.choice(candidates)
    east_pivot = find_farest(pivot, candidates)
    west_pivot = find_farest(east_pivot, candidates)
    c = cal_distance(east_pivot, west_pivot)

    if c == 0:
        east_items = candidates[:len(candidates)//2]
        west_items = candidates[len(candidates)//2:]
        return east_pivot, west_pivot, east_items, west_items

    all_distance = []
    for candidate in candidates:
        a = cal_distance(candidate, west_pivot)
        b = cal_distance(candidate, east_pivot)
        d = (a ** 2 + c ** 2 - b ** 2) / (2 * c)
        all_distance.append((d, candidate))

    all_distance.sort(key=lambda x: x[0])
    sorted_candidates = [item[1] for item in all_distance]
    east_items = sorted_candidates[:len(sorted_candidates)//2]
    west_items = sorted_candidates[len(sorted_candidates)//2:]

    return east_pivot, west_pivot, east_items, west_items


def find_farest(pivot, candidates):
    max_d = 0
    most_point = pivot

    for candidate in candidates:
        cur_d = cal_distance(pivot, candidate)
        if  cur_d > max_d:
            max_d = cur_d
            most_point = candidate

    return most_point


def cal_distance(p1, p2):
    return math.sqrt(sum([(v1 - v2) ** 2 for v1, v2 in zip(p1[:-1], p2[:-1])]))


def process_mixed_cluster(cluster):
    """
    in DE operation, use current-to-best to mutate the candidates
    v_i = x_i + F * (x_b - x_i) + F_min * (x_r1 - x_r2)
    """
    DE_params = {"F": 0.8, "Fmin": 0.1, "CR": 1.0}
    pos_point = [(idx, item) for idx, item in enumerate(cluster) if item[-1] == 1]
    neg_point = [(idx, item) for idx, item in enumerate(cluster) if item[-1] == 0]

    candidate_l = []
    if len(pos_point) == 1:
        # only 1 pos point in cluster, then mutate all neg points toward to the pos point
        xb = pos_point[0][1]
        R = random.choice(range(len(xb)-1))

        for _, xi in neg_point:
            new_candidate = []
            for i in range(len(xi)-1):
                ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                if ri < DE_params["CR"] or i == R:
                    new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]))
                else:
                    new_candidate.append(xi[i])

            new_candidate.append(1)
            candidate_l.append(np.array(new_candidate))
    else:
        # more than 2 pos points in cluster, then randomly pick 3 points, first 1 is current point, and
        # another two are support points
        for idx1, xb in pos_point:
            R = random.choice(range(len(xb)-1))
            for idx2, xi in enumerate(cluster):
                if not idx1 == idx2:
                    available_points = []
                    for idx3, p in enumerate(cluster):
                        if not idx3 == idx2 and not idx3 == idx1:
                            available_points.append(p)
                    [xr1, xr2] = random.sample(available_points, 2)

                    new_candidate = []
                    # if xr1 and xr2 all negative class, then just use xi
                    if xr1[-1] == 0 and xr2[-1] == 0:
                        for i in range(len(xi)-1):
                            ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                            if ri < DE_params["CR"] or i == R:
                                new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]))
                            else:
                                new_candidate.append(xi[i])
                    else:
                        for i in range(len(xi)-1):
                            ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                            if ri < DE_params["CR"] or i == R:
                                if xr1[-1] == 1:
                                    new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]) + DE_params["Fmin"] * (xr1[i] - xr2[i]))
                                else:
                                    new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]) + DE_params["Fmin"] * (xr2[i] - xr1[i]))
                            else:
                                new_candidate.append(xi[i])

                    new_candidate.append(1)
                    candidate_l.append(np.array(new_candidate))

    return candidate_l


def process_positive_cluster(cluster):
    """
    in DE operation, use best to mutate the candidates
    v_i = x_b + F * (x_r1 - x_r2)
    """
    DE_params = {"F": 0.8, "CR": 1.0}
    pos_point = [item for item in cluster if item[-1] == 1]
    candidate_l = []

    for idx1 in range(len(pos_point)-2):
        for idx2 in range(idx1+1, len(pos_point)-1):
            for idx3 in range(idx2+1, len(pos_point)):
                [xb, xr1, xr2] = random.sample([idx1, idx2, idx3], 3)
                xb, xr1, xr2 = pos_point[xb], pos_point[xr1], pos_point[xr2]

                R = random.choice(range(len(xb)-1))
                new_candidate = []

                for i in range(len(xb)-1):
                    ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                    if ri < DE_params["CR"] or i == R:
                        new_candidate.append(xb[i] + DE_params["F"] * (xr1[i] - xr2[i]))
                    else:
                        new_candidate.append(xb[i])

                new_candidate.append(1)
                candidate_l.append(np.array(new_candidate))

    return candidate_l


def process_mixed_cluster_extra(cluster):
    """
    in DE operation, use current-to-best-extra to mutate the candidates
    v_i = x_b + F * (x_r1 - x_r2) + F_ex * (x_r3 - x_r4)
    """
    DE_params = {"F": 0.8, "CR": 1.0, "F_xc": 0.1}
    pos_point = [item for item in cluster if item[-1] == 1]

    candidate_l = []
    for xb in pos_point:
        R = random.choice(range(len(xb)-1))

        for xi in cluster:
            if not np.array_equal(xb, xi):
                available_points = []
                for p in cluster:
                    if not np.array_equal(p, xi) and not np.array_equal(p, xb):
                        available_points.append(p)

                for _ in range(20):
                    [xr1, xr2, xr3, xr4] = random.sample(available_points, 4)

                    new_candidate = []

                    for i in range(len(xi)-1):
                        ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                        if ri < DE_params["CR"] or i == R:
                            new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]) + DE_params["F_xc"] * (xr1[i] - xr2[i]) + DE_params["F_xc"] * (xr3[i] - xr4[i]))
                        else:
                            new_candidate.append(xi[i])

                    new_candidate.append(1)
                    candidate_l.append(new_candidate)

    return candidate_l


def RandomProjectionOversampling(X_train, y_train, threshold):
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.reset_index(inplace=True, drop=True)
    tar = y_train.name
    X_train[tar] = y_train
    X_train.reset_index(inplace=True, drop=True)
    col_names = X_train.columns

    n_data_to_generate = X_train[tar].value_counts()[0] - X_train[tar].value_counts()[1]
    X_train = X_train.to_numpy()
    start_time = time.time()
    res = cluster(X_train, threshold, [])

    new_data_negative_cluster = []
    new_data_positive_cluster = []
    for c in res:
        if sum([item[-1] for item in c]) > len(c)//2:
            cur_new_data = process_positive_cluster(c)
            new_data_positive_cluster += cur_new_data
        else:
            cur_new_data = process_mixed_cluster(c)
            new_data_negative_cluster += cur_new_data

    rt = time.time() - start_time
    if len(new_data_negative_cluster) >= n_data_to_generate - len(new_data_positive_cluster):
        new_data = new_data_positive_cluster + random.sample(new_data_negative_cluster,
                                                            n_data_to_generate - len(new_data_positive_cluster))
    else:
        extra_data = []
        for c in res:
            cur_extra_data = process_mixed_cluster_extra(c)
            extra_data += cur_extra_data

        rest_data_to_generate = n_data_to_generate - len(new_data_positive_cluster) - len(new_data_negative_cluster)
        new_data = new_data_negative_cluster + new_data_positive_cluster + random.sample(extra_data, rest_data_to_generate)

    new_data_df = pd.DataFrame(np.array(new_data), columns=col_names)

    return rt, new_data_df, train_df

# SDV - Oversampling

In [ ]:
def do_sdv(X_train, y_train):
  train_df = pd.concat([X_train, y_train], axis=1)
  class_counts = y_train.value_counts()

  # Find minority class label
  minority_class_label = class_counts.idxmin()

  # Filter rows with minority class label
  minority_df = train_df[train_df.iloc[:, -1] == minority_class_label]

  # Calculate counts of majority and minority classes
  majority_count = class_counts.max()
  minority_count = class_counts.min()

  metadata_data = SingleTableMetadata()
  metadata_data.detect_from_dataframe(minority_df)
  # Generate synthetic data using GaussianCopulaSynthesizer
  synthesizer_breast_data = GaussianCopulaSynthesizer(metadata_data)
  synthesizer_breast_data.fit(minority_df)

  # Print sample synthetic data
  synthesizer_breast_data.reset_sampling()
  sd1 = synthesizer_breast_data.sample(num_rows=majority_count-minority_count)
  return sd1, train_df

# Function to add synthetic data to the main DataFrame based on percentage
def add_synthetic_data(main_df, synthetic_df, percentage, seed=42):
    # Calculate number of rows to sample
    num_rows = int(len(synthetic_df) * percentage)
    np.random.seed(seed)
    # Sample the specified percentage of synthetic data
    sampled_synthetic_data = synthetic_df.sample(n=num_rows, replace=False, random_state=seed)
    # print(sampled_synthetic_data)

    # Concatenate sampled synthetic data with main DataFrame
    combined_df = pd.concat([main_df, sampled_synthetic_data], ignore_index=True)
    # print(combined_df)
    return combined_df

# Random Over-Sampling

In [ ]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def random_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = RandomOverSampler(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# SMOTE Over-Sampling

In [ ]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def smote_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = SMOTE(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# SVM-SMOTE Over-Sampling

In [ ]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def svm_smote_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = SVMSMOTE(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# Intelligent Pruning

In [ ]:
def find_majority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    max_label = max(zip(counts, labels))[1]
    indices_with_max_label = np.where(y == max_label)[0]
    X_maj, y_maj = X[indices_with_max_label], y[indices_with_max_label]

    # Exclude majority class samples
    indices_without_max_label = np.where(y != max_label)[0]
    X_remaining, y_remaining = X[indices_without_max_label], y[indices_without_max_label]

    return X_maj, y_maj, X_remaining, y_remaining, min(counts)

def do_clustering(X, y, labels):
  clustered_X = defaultdict(list)
  clustered_y = defaultdict(list)

  for i, label in enumerate(labels):
      clustered_X[label].append(X[i])
      clustered_y[label].append(y[i])

  # Sort clustered_X and clustered_y in descending order based on the length of values in each dictionary
  sorted_clustered_X = dict(sorted(clustered_X.items(), key=lambda x: -len(x[1])))
  sorted_clustered_y = dict(sorted(clustered_y.items(), key=lambda x: -len(x[1])))

  return sorted_clustered_X, sorted_clustered_y


def intelligent_prune_data(pruning_samps, pruning_ratios, clustered_X, clustered_y, per_cluster_pruning_ratio=0.7, seed=42):
  random.seed(seed)
  pruning_ratios_X_maj, pruning_ratios_y_maj = defaultdict(list), defaultdict(list)
  for pruning_samp, pruning_ratio in zip(pruning_samps, pruning_ratios):
    samps = 0
    # print("For Pruning samps: ", pruning_samp)
    prune_samps = pruning_samp
    # print(prune_samps)
    clustered_X_new = defaultdict(list)
    clustered_y_new = defaultdict(list)
    # Iterate over the sorted dictionaries
    for label, values_X in clustered_X.items():
        # Calculate the number of samples to prune
        num_samples_to_prune = int(prune_samps * per_cluster_pruning_ratio)
        if(num_samples_to_prune > len(values_X)):
          num_samples_to_prune = len(values_X)//2
          prune_samps -= num_samples_to_prune
        else:
          prune_samps -= num_samples_to_prune

        # Randomly choose samples to prune
        indices_to_prune = random.sample(range(len(values_X)), num_samples_to_prune)

        # Prune the samples from clustered_X and clustered_y
        clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in indices_to_prune]
        clustered_y_new[label] = [clustered_y[label][i] for i in range(len(clustered_y[label])) if i not in indices_to_prune]

    iter = 0
    while(prune_samps > 0):
        if(iter>=100):
          break
        for label, values_X in clustered_X_new.items():
          if(prune_samps <=0 or len(values_X) <= 0):
            break
          # print(len(values_X))
          index_to_prune = random.sample(range(len(values_X)), 1)
          clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in index_to_prune]
          clustered_y_new[label] = [clustered_y_new[label][i] for i in range(len(clustered_y_new[label])) if i not in index_to_prune]

          prune_samps -= 1
        iter += 1

    for label in clustered_X_new:
        pruning_ratios_X_maj[pruning_ratio].extend(clustered_X_new[label])
        pruning_ratios_y_maj[pruning_ratio].extend(clustered_y_new[label])

  return pruning_ratios_X_maj, pruning_ratios_y_maj

def combine_data(pruning_ratios, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining):

  pruning_ratios_X, pruning_ratios_y = defaultdict(list), defaultdict(list)
  for pruning_ratio in pruning_ratios:
    pruning_ratios_X[pruning_ratio].extend(pruning_ratios_X_maj[pruning_ratio])
    pruning_ratios_X[pruning_ratio].extend(X_remaining)

    pruning_ratios_y[pruning_ratio].extend(pruning_ratios_y_maj[pruning_ratio])
    pruning_ratios_y[pruning_ratio].extend(y_remaining)

  return pruning_ratios_X, pruning_ratios_y

def do_intelligent_pruning(X, y, ratio, per_cluster_pruning_ratio=0.7, seed=42):

  X_maj, y_maj, X_remaining, y_remaining, min_class_samples = find_majority_data(X, y)
  kmeans = KMeans(n_clusters=3, random_state = 42)
  kmeans.fit(X_maj)
  labels = kmeans.labels_
  clustered_X, clustered_y = do_clustering(X_maj, y_maj, labels)

  pruning_best = len(X_maj)-min_class_samples
  pruning_samps = [int(pruning_best * ratio)]
  pruning_ratios = [ratio]

  pruning_ratios_X_maj, pruning_ratios_y_maj = intelligent_prune_data(pruning_samps, pruning_ratios, clustered_X, clustered_y, \
                                                                      per_cluster_pruning_ratio=per_cluster_pruning_ratio, seed=seed)

  pruning_ratios_X, pruning_ratios_y = combine_data(pruning_ratios, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining)

  return list(pruning_ratios_X.values()), list(pruning_ratios_y.values())

# Random Pruning

In [ ]:
"""
inputs:
X: np.array
y: np.array
percentage: from 0.0 upto 1.0, enter int value
"""
def random_prune_data(X, y, ratio, seed = 42):
  # preprocessed_X, scaler, imputer = preprocess_data_train(X)
  # preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

  # X_train, y_train = preprocessed_X_train.to_numpy(), y_train.to_numpy()
  # X_test, y_test = preprocessed_X_test.to_numpy(), y_test.to_numpy()
  np.random.seed(seed)
  labels_count = {}
  labels = np.unique(y)
  for label in labels:
    labels_count[label] = np.count_nonzero(y == label)
  max_label = min_label = labels[0]
  for label in labels_count:
    if labels_count[label] > labels_count[max_label]:
      max_label = label
    if labels_count[label] < labels_count[min_label]:
      min_label = label

  # print("Max", max_label, labels_count[max_label])
  # print("Min", min_label, labels_count[min_label])

  prune_counts = {}
  prune_indexes = {}
  for label in labels_count:
    prune_counts[label] = labels_count[label] - labels_count[min_label]
    prune_indexes[label] = np.where(y == label)[0]

  prune_amount = int(ratio * sum(map(lambda x: x[1], prune_counts.items())))
  prune_it = {}

  while prune_amount > 0:
    for label in labels:
      if (len(prune_indexes[label]) - labels_count[min_label]) > 0 and prune_amount > 0:
        random_index = np.random.choice(len(prune_indexes[label]))
        random_item = prune_indexes[label][random_index]
        prune_indexes[label] = np.delete(prune_indexes[label], random_index)
        if prune_it.get(label, None) is None:
          prune_it[label] = np.array([])
        prune_it[label] = np.append(prune_it[label], [random_item])
        prune_amount -= 1



  formatted_indexes = np.array([])
  for label in prune_indexes:
    formatted_indexes = np.append(formatted_indexes, prune_indexes[label])
  formatted_indexes = np.sort(formatted_indexes)
  new_arr = np.array([np.int64(i) for i in formatted_indexes])

  return X[new_arr], y[new_arr]

In [ ]:
def apply_smo(X, y, smo_K, random_state, budget=10):

    # Get indices for each class
    np.random.seed(random_state)
    class_indices = [np.where(y == i)[0] for i in np.unique(y)]
    # Shuffle class_indices
    for indices in class_indices:
        np.random.shuffle(indices)

    # Select K samples for each class
    selected_indices = [indices[:smo_K] for indices in class_indices]

    # Flatten the list of indices
    selected_indices_flat = np.concatenate(selected_indices)

    # Get the selected data and target values
    selected_data = X.loc[selected_indices_flat]
    selected_target = y.loc[selected_indices_flat]

    selected_data_df = pd.DataFrame(selected_data, columns=X.columns)
    selected_target_df = pd.Series(selected_target, name=y.name)

    # Store the remaining samples as well
    remaining_indices_flat = np.setdiff1d(np.arange(len(X)), selected_indices_flat)
    remaining_data = X.loc[remaining_indices_flat]
    remaining_target =  y.loc[remaining_indices_flat]

    clf = GaussianNB()
    clf.fit(selected_data, selected_target)

    for i in range(budget):
        out, thresh, out_label = 0, 1E-30, 0
        for index, row in remaining_data.iterrows():
            target_value = remaining_target.loc[index]
            row_2d = row.values.reshape(1, -1)
            p1, p2 = clf.predict_proba(row_2d)[0][0], clf.predict_proba(row_2d)[0][1]
            score = abs(p1+p2)/(abs(p1-p2)+1E-300)
            if(score > thresh):
                out = index
                thresh = score
                if(p1<p2):
                    out_label = 1
                else:
                    out_label = 0

        X_to_move = (remaining_data.loc[out]).values.reshape(1, -1)
        y_to_move = [out_label]

        # Append the row to remaining_data
        X_to_move_df = pd.DataFrame(X_to_move, columns=remaining_data.columns)
        selected_data_df = pd.concat([selected_data_df, X_to_move_df], ignore_index=True)

        # Append the value to remaining_target
        y_to_move_series = pd.Series(y_to_move, name=remaining_target.name)
        selected_target_df = pd.concat([selected_target_df, y_to_move_series], ignore_index=True)

        clf.partial_fit(X_to_move, y_to_move)

        remaining_data = remaining_data.drop(out).reset_index(drop=True)
        remaining_target = remaining_target.drop(out).reset_index(drop=True)


    remaining_data_df = pd.DataFrame(remaining_data, columns=X.columns)

    remaining_target_df = pd.Series(clf.predict(remaining_data_df), name=remaining_target.name)
    matches = (remaining_target_df == remaining_target).sum()
    X_new = pd.concat([selected_data_df, remaining_data_df])
    y_new = pd.concat([selected_target_df, remaining_target_df])

    return X_new, y_new, matches, len(remaining_target_df)

In [ ]:
ratios = [ratio for ratio in np.arange(0, 1.1, 0.2)]

In [ ]:
learners = {'Logistic Regression' : 'LR', 'SVM': 'SVM', 'Decision Tree': 'DT' }

df = pd.concat([train_df, test_df], axis=0)

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

states = [82, 15, 4, 95, 36, 32, 29, 18, 14, 87]
ratios = [0.2, 0.4, 0.6, 0.8, 1.0]
smo_K = 5
smo = True

In [ ]:
def split_data(X, y, random_state):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state)
  return X_train, X_test, y_train, y_test

In [ ]:
smo_X_train, smo_y_train = dict(), dict()
for rand_state in states:
  X_train, X_test, y_train, y_test = split_data(X, y,rand_state)
  X_train.reset_index(drop=True, inplace=True)
  X_test.reset_index(drop=True, inplace=True)
  y_train.reset_index(drop=True, inplace=True)
  y_test.reset_index(drop=True, inplace=True)
  X_train, y_train, matches, total = apply_smo(X_train, y_train, smo_K, rand_state)

  smo_X_train[rand_state] = X_train
  smo_y_train[rand_state] = y_train
  print(str(matches) + " Matches out of " + str(total) + " dark samples")

# Calling Intelligent Pruning

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'intelligent_pruning'
file_name = f'{algorithm}_results'
per_cluster_pruning_ratios = [0.5, 0.7, 0.9, 1]

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for per_cluster_pruning_ratio in per_cluster_pruning_ratios:
    for ratio in ratios:

      X_train_copy, y_train_copy = X_train.copy(), y_train.copy()

      intelligent_pruned_X_train, intelligent_pruned_y_train = do_intelligent_pruning(X_train_copy.to_numpy(), y_train_copy.to_numpy(), ratio, per_cluster_pruning_ratio=per_cluster_pruning_ratio)

      preprocessed_intelligent_pruned_X_train, scaler, imputer = preprocess_data_train((np.array(intelligent_pruned_X_train))[0])
      preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

      intelligent_pruned_X_train, intelligent_pruned_y_train = preprocessed_intelligent_pruned_X_train, (np.array(intelligent_pruned_y_train))[0]
      intelligent_pruned_X_test, intelligent_pruned_y_test = preprocessed_X_test, y_test.to_numpy()
      results = evaluate_models(intelligent_pruned_X_train, intelligent_pruned_X_test, intelligent_pruned_y_train, intelligent_pruned_y_test, rand_state)
      for key, item in results.items():
        row = {'algo' : [algorithm], 'ratio': [ratio], 'per_cluster_pruning_ratio': [per_cluster_pruning_ratio],'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
        temp = pd.DataFrame(row)
        df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has 

#Calling Random Pruning

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'random_pruning'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:

    random_pruned_X_train, random_pruned_y_train = random_prune_data(X_train.to_numpy(), y_train.to_numpy(), ratio, seed=rand_state)
    preprocessed_random_pruned_X_train, scaler, imputer = preprocess_data_train(random_pruned_X_train)
    preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

    random_pruned_X_train, random_pruned_y_train = preprocessed_random_pruned_X_train, random_pruned_y_train
    random_pruned_X_test, random_pruned_y_test = preprocessed_X_test, y_test.to_numpy()

    print(f"Train data pruned randomly at {ratio * 100}%")
    results = evaluate_models(random_pruned_X_train, random_pruned_X_test, random_pruned_y_train, random_pruned_y_test, rand_state)
    for key, item in results.items():
      row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
      temp = pd.DataFrame(row)
      df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned randomly at 20.0%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned randomly at 40.0%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


...

Train data pruned randomly at 60.0%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned randomly at 80.0%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data pruned randomly at 100.0%


# Calling SDV-Oversampling

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'gaussian_copula'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:

    sd1, train_df = do_sdv(X_train.drop(columns=['state']), y_train)
    results_syn_sdv = dict()

    # Add synthetic data at different percentages to the main DataFrame
    combined_df = add_synthetic_data(train_df, sd1, ratio, seed=rand_state)
    y_train_sdv = combined_df.iloc[:,-1]
    X_train_sdv = combined_df.iloc[:,:-1]

    preprocessed_X_train_sdv, scaler, imputer = preprocess_data_train(X_train_sdv)
    preprocessed_X_test_sdv = preprocess_data_test(X_test.drop(columns=['state']), scaler, imputer)

    X_train_sdv, y_train_sdv = preprocessed_X_train_sdv, y_train_sdv.to_numpy()
    X_test_sdv, y_test_sdv = preprocessed_X_test_sdv, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class")
    results = evaluate_models(X_train_sdv, X_test_sdv, y_train_sdv, y_test_sdv, rand_state)
    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:80: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Train data combined with 20.0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:80: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Train data combined with 40.0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:80: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:80: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Train data combined with 80.0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:80: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Train data combined with 100.0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Calling SMOTE-Oversampling

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'smote'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:

    X_train_smote, y_train_smote = smote_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])
    preprocessed_X_train_smote, scaler, imputer = preprocess_data_train((np.array(X_train_smote))[0])
    preprocessed_X_test_smote = preprocess_data_test(X_test, scaler, imputer)

    X_train_smote, y_train_smote = preprocessed_X_train_smote, (np.array(y_train_smote))[0]
    X_test_smote, y_test_smote = preprocessed_X_test_smote, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    results = evaluate_models(X_train_smote, X_test_smote, y_train_smote, y_test_smote, rand_state)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 20.0% synthetic data of minority class:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 40.0% synthetic data of minority class:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


...

Train data combined with 60.0% synthetic data of minority class:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 80.0% synthetic data of minority class:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


Train data combined with 100.0% synthetic data of minority class:


# Calling Random-Oversampling

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'random_oversampling'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:

    X_train_random, y_train_random = random_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])

    preprocessed_X_train_random, scaler, imputer = preprocess_data_train((np.array(X_train_random)[0]))
    preprocessed_X_test_random = preprocess_data_test(X_test, scaler, imputer)

    X_train_random, y_train_random = preprocessed_X_train_random, (np.array(y_train_random))[0]
    X_test_random, y_test_random = preprocessed_X_test_random, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    results = evaluate_models(X_train_random, X_test_random, y_train_random, y_test_random, rand_state)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

# Calling SVM-SMOTE Over-Sampling

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'svm_smote'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:

    X_train_svm_smote, y_train_svm_smote = svm_smote_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])

    preprocessed_X_train_svm_smote, scaler, imputer = preprocess_data_train((np.array(X_train_svm_smote))[0])
    preprocessed_X_test_svm_smote = preprocess_data_test(X_test, scaler, imputer)

    X_train_svm_smote, y_train_svm_smote = preprocessed_X_train_svm_smote, (np.array(y_train_svm_smote))[0]
    X_test_svm_smote, y_test_svm_smote = preprocessed_X_test_svm_smote, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    results = evaluate_models(X_train_svm_smote, X_test_svm_smote, y_train_svm_smote, y_test_svm_smote, rand_state)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

# Calling for RRP Over-Sampling

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'RRP'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:

    X_train_copy, y_train_copy = X_train.copy(), y_train.copy()

    rt, new_data_df, train_df = RandomProjectionOversampling(X_train=X_train_copy,
                                                                        y_train=y_train_copy,
                                                                        threshold=20
                                                             )

    # Add synthetic data at different percentages to the main DataFrame
    combined_df = add_synthetic_data(train_df, new_data_df, ratio, seed=rand_state)
    y_train_rrp = combined_df.iloc[:,-1]
    X_train_rrp = combined_df.iloc[:,:-1]

    preprocessed_X_train_rrp, scaler, imputer = preprocess_data_train(X_train_rrp)
    preprocessed_X_test_rrp = preprocess_data_test(X_test, scaler, imputer)

    X_train_rrp, y_train_rrp = preprocessed_X_train_rrp, y_train_rrp.to_numpy()
    X_test_rrp, y_test_rrp = preprocessed_X_test_rrp, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class")
    results = evaluate_models(X_train_rrp, X_test_rrp, y_train_rrp, y_test_rrp, rand_state)
    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

Train data combined with 20.0% synthetic data of minority class
Train data combined with 40.0% synthetic data of minority class
Train data combined with 60.0% synthetic data of minority class
Train data combined with 80.0% synthetic data of minority class
Train data combined with 100.0% synthetic data of minority class
Train data combined with 20.0% synthetic data of minority class
Train data combined with 40.0% synthetic data of minority class
Train data combined with 60.0% synthetic data of minority class
Train data combined with 80.0% synthetic data of minority class
Train data combined with 100.0% synthetic data of minority class
Train data combined with 20.0% synthetic data of minority class
Train data combined with 40.0% synthetic data of minority class
Train data combined with 60.0% synthetic data of minority class
Train data combined with 80.0% synthetic data of minority class
Train data combined with 100.0% synthetic data of minority class
Train data combined with 20.0% synthe

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train data combined with 40.0% synthetic data of minority class
Train data combined with 60.0% synthetic data of minority class
Train data combined with 80.0% synthetic data of minority class
Train data combined with 100.0% synthetic data of minority class
Train data combined with 20.0% synthetic data of minority class
Train data combined with 40.0% synthetic data of minority class
Train data combined with 60.0% synthetic data of minority class
Train data combined with 80.0% synthetic data of minority class
Train data combined with 100.0% synthetic data of minority class
Train data combined with 20.0% synthetic data of minority class
Train data combined with 40.0% synthetic data of minority class
Train data combined with 60.0% synthetic data of minority class
Train data combined with 80.0% synthetic data of minority class
Train data combined with 100.0% synthetic data of minority class
Train data combined with 20.0% synthetic data of minority class
Train data combined with 40.0% synthe

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train data combined with 40.0% synthetic data of minority class
Train data combined with 60.0% synthetic data of minority class
Train data combined with 80.0% synthetic data of minority class
Train data combined with 100.0% synthetic data of minority class


# No-Sampling

In [ ]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'No_Sampling'
file_name = f'{algorithm}_results'

ratios = [0]
for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:

    X_train_no_sampling, y_train_no_sampling = X_train.copy(), y_train.copy()

    preprocessed_X_train_no_sampling, scaler, imputer = preprocess_data_train(X_train_no_sampling)
    preprocessed_X_test_no_sampling = preprocess_data_test(X_test, scaler, imputer)

    X_train_no_sampling, y_train_no_sampling = preprocessed_X_train_no_sampling, y_train_no_sampling.to_numpy()
    X_test_no_sampling, y_test_no_sampling = preprocessed_X_test_no_sampling, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class")
    results = evaluate_models(X_train_no_sampling, X_test_no_sampling, y_train_no_sampling, y_test_no_sampling, rand_state)
    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

Train data combined with 0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train data combined with 0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train data combined with 0% synthetic data of minority class


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train data combined with 0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train data combined with 0% synthetic data of minority class


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
